In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import ForeignKey


In [2]:
# DB接続
engine = create_engine('sqlite:///test.db')

In [3]:
# Base
Base = declarative_base()

In [4]:
type(Base)

sqlalchemy.orm.decl_api.DeclarativeMeta

In [5]:
# テーブルクラス定義
class User(Base):
    """Usersテーブル

    Args:
        Base (sqlalchemy.orm.decl_api.DeclarativeMeta): ormのベース
    """

    # テーブル名
    __tablename__ = 'users'

    # 個々のカラムを定義
    id = Column(Integer, primary_key=True)
    name = Column(String)
    age = Column(Integer)

In [6]:
class Post(Base):
    """Postテーブル

    Args:
        Base (sqlalchemy.orm.decl_api.DeclarativeMeta): ormのベース
    """

    # テーブル名
    __tablename__ = 'posts'

    # 個々のカラムを定義
    id = Column(Integer, primary_key=True)
    users_id = Column(Integer, ForeignKey('users.id'))
    title = Column(String)
    body = Column(Integer)

In [7]:
# テーブルクラスのテーブルを生成
Base.metadata.create_all(engine)

In [8]:
# セッション生成
Session = sessionmaker(bind=engine)
session = Session()

In [9]:
# サンプルデータ挿入
session.add(User(id=1, name="Suzuki", age=19))
session.add(User(id=2, name="Tanaka", age=21))
session.add(User(id=3, name="Sato", age=21))

session.add(Post(users_id=1, title="朝の体操", body="ラジオ体操で元気いっぱい"))
session.add(Post(users_id=1, title="今日の夕食", body="カレーラスがとても美味しかった。"))
session.add(Post(users_id=2, title="仕事", body="今日はDjangoでAPI作成。"))
session.add(Post(users_id=2, title="Python楽しい", body="Python楽しいですよね！！"))
session.commit()


In [10]:
# inner joinのサンプル
users_posts = session.query(User, Post).join(
    Post, User.id == Post.users_id).all()

for user_post in users_posts:
    print(f'{user_post.User.name}さんの投稿 タイトル: {user_post.Post.title}')


Suzukiさんの投稿 タイトル: 朝の体操
Suzukiさんの投稿 タイトル: 今日の夕食
Tanakaさんの投稿 タイトル: 仕事
Tanakaさんの投稿 タイトル: Python楽しい


In [11]:
# left outer joinのサンプル
users_posts = session.query(User, Post).outerjoin(
    Post, User.id == Post.users_id).all()

for user_post in users_posts:
    if user_post.Post is not None:
        print(f'{user_post.User.name}さんの投稿 タイトル: {user_post.Post.title}')
    else:
        print(f'{user_post.User.name}さんの投稿 なし')

Suzukiさんの投稿 タイトル: 今日の夕食
Suzukiさんの投稿 タイトル: 朝の体操
Tanakaさんの投稿 タイトル: Python楽しい
Tanakaさんの投稿 タイトル: 仕事
Satoさんの投稿 なし
